In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

%matplotlib inline  

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Datas

In [3]:
''' Колонки '''
tick = '^GSPC'
ma1 = 30
ma2 = 150


''' параметры анализа '''
an_columns = 3
deep = 6
type_str = 'vertical_more_or_low'

''' время '''
stime='2021-01-01'
ftime=None
period = '1d'  # '1d' '1wk'

Добавляем тикер и изменение

In [4]:
def get_data_from_ticker(tick, stime, ftime, interval=period):
    ticker = yf.Ticker(tick)

    df = ticker.history(start=stime, end=ftime, interval=interval)
    x = pd.DataFrame(df)
    x.rename(columns={"Close": tick}, inplace=True)
    z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
    return z

df = pd.DataFrame(get_data_from_ticker(tick, stime, ftime))


In [5]:
def add_percent_update(tick):
    """ date stock's or market's update"""
    gf_copy = df.copy(deep=True)
    gf_copy = pd.DataFrame(gf_copy.drop(df.index[0]))
    gf_copy_v = gf_copy[tick].values
    gf_v = df.copy(deep=True)
    gf_vv = gf_v[tick].values

    for i in range(len(gf_copy)):
        gf_vv[i] = (gf_copy_v[i] - gf_vv[i]) / (gf_vv[i] / 100)

    gf_vv = np.insert(gf_vv, 0, 0)
    gf_vv = np.delete(gf_vv, -1)
    return gf_vv

df[f"% update {tick}"] = add_percent_update(tick)
# gf[f"% update {st}"] = add_percent_update(st)

# df = df[[f"% update {tick}", tick]]
df.head(3)

,^GSPC,% update ^GSPC
Date,,
2020-12-28,3756.070068,0.000000
2021-01-04,3824.679932,1.826640
2021-01-11,3768.250000,-1.475416


Добавляем MA 

In [6]:
def add_ma(df, num):
    x = df.columns[0]
    df[f'ma_{num}'] = df[x].ewm(span=num, adjust=False).mean()
    return df    

try:
    df = add_ma(df, ma1)
    df = add_ma(df, ma2)
except:
    df = add_ma(df, ma1)
finally:
    pass

df.head(5)

,^GSPC,% update ^GSPC,ma_30,ma_150
Date,,,,
2020-12-28,3756.070068,0.000000,3756.070068,3756.070068
2021-01-04,3824.679932,1.826640,3760.496511,3756.978808
2021-01-11,3768.250000,-1.475416,3760.996736,3757.128096
2021-01-18,3841.469971,1.943076,3766.188558,3758.245207
2021-01-25,3714.239990,-3.312013,3762.837037,3757.662356


Get consistent !!!

In [7]:
# l= []
# for i in range(len(df[tick])):
#     if i <= deep:
#         for y in range(3):
#             if y == 0: 
#                 pass
#             elif y > 0:
#                 if df.iloc[i-y, 1] > df.iloc[i-1-y, 3]:
#                     x = 'm'
#                     l.append(x)
#                 else:
#                     x = 'l'
#                     l.append(x)
#             else:
#                 pass
#     else:
#         pass
        
# print[l]        



In [8]:


def analyse_f(df, type_str, deep, num_col):
    deep += 1
    # r = list(df.iloc[:,0].values)
    r =  []
    if type_str == 'vertical_more_or_low':
        for i in range(len(df[tick])):
            if i <= deep:
                r.append(0)
            else:    
                l = []
                for y in range(deep):
                    if y == 0: 
                        pass
                    elif y > 0:
                        if df.iloc[i-y, num_col] > df.iloc[i-1-y, num_col]:
                            x = 'm'
                            l.append(x)
                        else:
                            x = 'l'
                            l.append(x)
                    else:
                        pass
                r.append(l)
    else:
        pass 
    return r
        
# !!! добавить функцию для обработки нескольких столбцов
def columns_for_an(df, type_str, deep, an_columns):
    for i in range(1, an_columns+1):
        df[f'consistent_{i}'] = analyse_f(df, type_str, deep, i)
    return df
        
        
df = columns_for_an(df, type_str, deep, an_columns)        


df.tail(20)        

,^GSPC,% update ^GSPC,ma_30,ma_150,consistent_1,consistent_2,consistent_3
Date,,,,,,,
2022-08-15,4228.479980,-1.207199,4164.728661,4101.161933,"[m, l, m, m, l, m]","[m, l, l, l, l, l]","[m, m, m, l, l, l]"
2022-08-22,4057.659912,-4.039751,4157.821000,4100.585748,"[l, m, l, m, m, l]","[m, m, l, l, l, l]","[m, m, m, m, l, l]"
2022-08-29,3924.260010,-3.287607,4142.752549,4098.250307,"[l, l, m, l, m, m]","[l, m, m, l, l, l]","[l, m, m, m, m, l]"
2022-09-05,4067.360107,3.646550,4137.888520,4097.841166,"[m, l, l, m, l, m]","[l, l, m, m, l, l]","[l, l, m, m, m, m]"
2022-09-12,3873.330078,-4.770417,4120.820234,4094.867509,"[m, m, l, l, m, l]","[l, l, l, m, m, l]","[l, l, l, m, m, m]"
2022-09-19,3693.229980,-4.649748,4093.233766,4089.547806,"[l, m, m, l, l, m]","[l, l, l, l, m, m]","[l, l, l, l, m, m]"
2022-09-26,3585.620117,-2.913706,4060.484498,4082.873268,"[m, l, m, m, l, l]","[l, l, l, l, l, m]","[l, l, l, l, l, m]"
2022-10-03,3639.659912,1.507125,4033.334525,4077.002892,"[m, m, l, m, m, l]","[l, l, l, l, l, l]","[l, l, l, l, l, l]"
2022-10-10,3583.070068,-1.554811,4004.285205,4070.460735,"[m, m, m, l, m, m]","[l, l, l, l, l, l]","[l, l, l, l, l, l]"


Получаем колонки для анализа

In [9]:
column_names = []
for v in df.columns:
    if 'consistent_' in v: 
        column_names.append(v)
    else:
        pass
    
column_names

['consistent_1', 'consistent_2', 'consistent_3']

In [10]:
def concat_cons_column(df, column_names):
    ''' del Zero '''
    df.loc[df['consistent_1'] != 0]
    
    ''' concat '''
    
    df['concat_cons'] = df[column_names[0]].astype(str)
    for i in range(len(column_names)):
        if i > 0:
            df['concat_cons'] = df['concat_cons'] + df[column_names[i]].astype(str)
        else:
            pass
    return df
                   
df = concat_cons_column(df, column_names)
df.tail(3)

,^GSPC,% update ^GSPC,ma_30,ma_150,consistent_1,consistent_2,consistent_3,concat_cons
Date,,,,,,,,
2022-12-12,3852.360107,-2.084694,3968.974426,4053.471474,"[l, l, m, l, m, l]","[l, m, m, l, m, l]","[l, m, l, l, l, l]","['l', 'l', 'm', 'l', 'm', 'l']['l', 'm', 'm', ..."
2022-12-19,3844.820068,-0.195725,3960.964467,4050.707879,"[m, l, l, m, l, m]","[l, l, m, m, l, m]","[l, l, m, l, l, l]","['m', 'l', 'l', 'm', 'l', 'm']['l', 'l', 'm', ..."
2022-12-26,3839.500000,-0.138370,3953.128050,4047.910424,"[m, m, l, l, m, l]","[l, l, l, m, m, l]","[l, l, l, m, l, l]","['m', 'm', 'l', 'l', 'm', 'l']['l', 'l', 'l', ..."


Поиск наиболее эффективных значений

In [11]:

''' Выбираем уникальные '''

df_un = df['concat_cons'].copy()
df_un = df_un.drop_duplicates()
set_consistens = list(df_un)

''' Фильтруем по типам и получаем их суммы '''
r = []
for i in range(len(set_consistens)):
    x = df.loc[(df['concat_cons'] == set_consistens[i])]
    r.append(f"Сумма {set_consistens[i]} = {x[f'% update {tick}'].sum()}")

for i in r:
    print(i)
    
# for i in range(len(column_names)):
#     get_effective_values(df, column_names[i], tick)

Сумма 000 = 4.149293797177192
Сумма ['l', 'l', 'm', 'l', 'm', 'l']['m', 'm', 'm', 'l', 'm', 'm']['m', 'm', 'm', 'l', 'm', 'm'] = -2.446049477674004
Сумма ['l', 'l', 'l', 'm', 'l', 'm']['m', 'm', 'm', 'm', 'l', 'm']['m', 'm', 'm', 'm', 'l', 'm'] = 0.8078936764876393
Сумма ['m', 'l', 'l', 'l', 'm', 'l']['m', 'm', 'm', 'm', 'm', 'l']['m', 'm', 'm', 'm', 'm', 'l'] = 2.6392954609087385
Сумма ['m', 'm', 'l', 'l', 'l', 'm']['m', 'm', 'm', 'm', 'm', 'm']['m', 'm', 'm', 'm', 'm', 'm'] = -0.7668623441137435
Сумма ['l', 'm', 'm', 'l', 'l', 'l']['m', 'm', 'm', 'm', 'm', 'm']['m', 'm', 'm', 'm', 'm', 'm'] = 1.5701091173990014
Сумма ['m', 'l', 'm', 'm', 'l', 'l']['m', 'm', 'm', 'm', 'm', 'm']['m', 'm', 'm', 'm', 'm', 'm'] = 4.123979375219285
Сумма ['l', 'm', 'l', 'm', 'm', 'l']['m', 'm', 'm', 'm', 'm', 'm']['m', 'm', 'm', 'm', 'm', 'm'] = 0.839302451459814
Сумма ['m', 'l', 'm', 'l', 'm', 'm']['m', 'm', 'm', 'm', 'm', 'm']['m', 'm', 'm', 'm', 'm', 'm'] = 1.3725637676089566
Сумма ['l', 'm', 'l', 'm', 

In [12]:
# ['l', 'm', 'l']['l', 'l', 'l']['l', 'l', 'l']

In [13]:
x = range(1, 3)
for n in x:
  print(n)

1
2
